In [1]:
!pip install langchain langchain_community deepeval

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of ragas to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.1/384.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install langchain-google-genai

In [4]:
from deepeval import assert_test
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric
import google.generativeai as genai
from deepeval.models.base_model import DeepEvalBaseLLM
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAI,GoogleGenerativeAIEmbeddings

In [5]:
GOOGLE_API_KEY = "AIzaSyCvDhMJK9JCVev9GSrgOSh-_GgwEqz"
genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
model = ChatGoogleGenerativeAI(model = "gemini-pro",google_api_key= "AIzaSyCvDhMJK9JCVev9GSrgOSh-_GgwEqzOEx")
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key="AIzaSyCvDhMJK9JCVev9GSrgOSh-_GgwEqzOU")

In [7]:
class GoogleVertexaI(DeepEvalBaseLLM):
    def __init__(self,model):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self,prompt):
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self,prompt):
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    def get_model_name(self):
        return "Vertex AI Model"


In [8]:
vertexai_gemini = GoogleVertexaI(model=model)

In [9]:
print(vertexai_gemini.generate("Write a poem on Lord Siva"))

**Ode to Lord Siva**

In realms beyond, where shadows dance,
Stands Lord Siva, a cosmic trance.
His matted locks, a sacred crown,
Adorned with moon, a celestial gown.

His third eye, a piercing light,
Illuminates darkness, dispels the night.
Around his neck, a serpent coiled,
Wisdom and power, forever coiled.

Blue as the heavens, his skin so fair,
A symbol of truth beyond compare.
He holds a trident, mighty and strong,
Protecting realms from right to wrong.

As Nataraja, he dances with grace,
Creating, destroying, in cosmic embrace.
His rhythm echoes through the void,
A symphony of life, forever deployed.

He is the destroyer, the transformer so grand,
Who ends cycles, with a mighty hand.
Yet within destruction, seeds of creation lie,
As the old gives way, the new to fly.

Lord Siva, the cosmic force supreme,
Whose presence pervades every dream.
In meditation deep, he seeks the truth,
Guiding souls through life's winding ruth.

Oh, Lord of the mountains, Lord of the dance,
May your bl

In [10]:
test_case = LLMTestCase(input="Python vs R? What is Better",actual_output="Python is best")

relevancy_metric = AnswerRelevancyMetric(threshold=0.7,model=vertexai_gemini)

In [11]:
relevancy_metric.measure(test_case)
print(relevancy_metric.score,relevancy_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0 The score is 1.00 because the actual output contains no irrelevant statements.


In [12]:
from deepeval.test_case import LLMTestCase,LLMTestCaseParams

from deepeval.metrics import GEval

In [15]:
testcases = LLMTestCase(input="What is better?Python or R",actual_output="Python is better",expected_output="Python")

correctness_metric = GEval(
    name="correctness",
    criteria="correctness - determine if output is short or not",
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT,LLMTestCaseParams.EXPECTED_OUTPUT],
    model=vertexai_gemini
)

In [16]:
correctness_metric.measure(testcases)
print(correctness_metric.score,correctness_metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

1.0 Actual Output is shorter than Expected Output, so it's classified as 'short'.


In [21]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=79783943d47f3ad35f1b71b57f6c3ca66743600dccf41958d452c06157deb901
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [22]:
from rouge_score import rouge_scorer

In [17]:
from deepeval.scorer import Scorer
from deepeval.metrics import BaseMetric

In [23]:
from deepeval.scorer import Scorer
from deepeval.metrics import BaseMetric

class RougeMetric(BaseMetric):
    def __init__(self, threshold: float = 0.5):
        self.threshold = threshold
        self.scorer = Scorer()

    def measure(self, test_case: LLMTestCase):
        self.score = self.scorer.rouge_score(
            prediction=test_case.actual_output,
            target=test_case.expected_output,
            score_type="rougeL"
        )
        self.success = self.score >= self.threshold
        return self.score

    # Async implementation of measure(). If async version for
    # scoring method does not exist, just reuse the measure method.
    async def a_measure(self, test_case: LLMTestCase):
        return self.measure(test_case)

    def is_successful(self):
        return self.success

    @property
    def __name__(self):
        return "Rouge Metric"


In [24]:
test_case = LLMTestCase(input="Is Python better than R",actual_output="Yes it is",expected_output="Yes")

metric = RougeMetric()

In [25]:
metric.measure(test_case=test_case)

0.5

In [26]:
from deepeval import evaluate
from deepeval.metrics import HallucinationMetric,AnswerRelevancyMetric
from deepeval.dataset import EvaluationDataset
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset


In [27]:
"""test_case = LLMTestCase(
  input="Who is the current president of the United States of America?",
  actual_output="Joe Biden",
  retrieval_context=["Joe Biden serves as the current president of America."]
)"""

first_test_case = LLMTestCase(input = "What is the capital of India",actual_output="Mumbai",expected_output="New Delhi")
second_test_case = LLMTestCase(input="Who is the father of Nation",actual_output="Gandhi",expected_output="Mahatma Gandhi")

test_case= [first_test_case,second_test_case]
dataset = EvaluationDataset(test_cases= test_case)


In [28]:
answer_relevancy = AnswerRelevancyMetric(model=vertexai_gemini)

In [30]:
evaluate(test_case,[answer_relevancy])

✨ You're running DeepEval's latest Answer Relevancy Metric! (using Vertex AI Model, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 2 test case(s) in parallel: |██████████|100% (2/2) [Time Taken: 00:05,  2.87s/test case]



Metrics Summary

  - ❌ Answer Relevancy (score: 0.0, threshold: 0.5, strict: False, evaluation model: Vertex AI Model, reason: The score is 0.00 because the statement made in the actual output is completely irrelevant to the input, which asks about the capital of India., error: None)

For test case:

  - input: What is the capital of India
  - actual output: Mumbai
  - expected output: New Delhi
  - context: None
  - retrieval context: None


Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: Vertex AI Model, reason: The score is 1.00 because the answer is fully relevant to the input., error: None)

For test case:

  - input: Who is the father of Nation
  - actual output: Gandhi
  - expected output: Mahatma Gandhi
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

Answer Relevancy: 50.00% pass rate




✓ Tests finished 🎉! Run 'deepeval login' to view evaluation results on Confident AI. 
‼️  NOTE: You can also run evaluations on ALL of deepeval's metrics directly on Confident AI instead.

[TestResult(success=False, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=False, score=0.0, reason='The score is 0.00 because the statement made in the actual output is completely irrelevant to the input, which asks about the capital of India.', strict_mode=False, evaluation_model='Vertex AI Model', error=None, evaluation_cost=None, verbose_logs='Statements:\n[\n    "Mumbai"\n] \n \nVerdicts:\n[\n    {\n        "verdict": "no",\n        "reason": "The statement made in the actual output is completely irrelevant to the input, which asks about the capital of India."\n    }\n]')], conversational=False, multimodal=False, input='What is the capital of India', actual_output='Mumbai', expected_output='New Delhi', context=None, retrieval_context=None),
 TestResult(success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because the answer is fully relevant to the input.', strict_mode=False, evalu